## Faster interactive exploration of 1 billion points
This example applies this `HilbertFrame2D` to the 1 billion point Open Street Maps dataset.

First the dataset must be preprocessed.  This is an intesive operation that took ~30 minutes on a 2015 Macbook Pro with 16GB of RAM.  It also seems to require something like 20GB of free disk space available in order to perform the shuffle sort of the Hilbert distance index.

In [ ]:
import dask
import datashader as ds
import datashader.transfer_functions as tf
import dask.dataframe as dd
import numpy as np
import hilbert_frame as hf

In [ ]:
df = dd.io.parquet.read_parquet('../../../notebook/datashader_dev/datashader-examples/data/osm-1billion.snappy.parq/')

In [ ]:
%%time
hframe = hf.HilbertFrame2D.from_dataframe(df=df,
                                          filename='osm-1billion.parquet',
                                          x='x',
                                          y='y',
                                          shuffle='disk',
                                          compression='snappy',
                                          engine='fastparquet')

Load the `HilbertFrame2D` with `persist=True` to bring the dataframe into memory

In [ ]:
hframe = hf.HilbertFrame2D(filename='osm-1billion.parquet', persist=True)

Create a Bokeh `InteractiveImage` for exploring the dataset, and notice how much faster the render time is when zoomed in to detailed regions around the world.

In [ ]:
bound = 20026376.39
bounds = dict(x_range = (-bound, bound), y_range = (int(-bound*0.4), int(bound*0.6)))
plot_width = 900
plot_height = int(plot_width*0.5)

In [ ]:
def create_image(use_hilbert, x_range, y_range, w, h, name=None):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    df_cb = hframe.range_query(x_range, y_range)
    agg = cvs.points(df_cb, 'x', 'y', ds.count())
    img = tf.shade(agg)
    return img

In [ ]:
from datashader.bokeh_ext import InteractiveImage
import bokeh.plotting as bp
from functools import partial

bp.output_notebook()
p = bp.figure(tools='pan,wheel_zoom,reset,box_zoom', x_range=bounds['x_range'], y_range=bounds['y_range'],
              plot_width=plot_width, plot_height=plot_height)

InteractiveImage(p, partial(create_image, True))

### Interaction GIF
![](osm-one-billion.gif)